In [0]:
spark.conf.set("fs.azure.account.auth.type.streamingsa.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.streamingsa.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.streamingsa.dfs.core.windows.net", {token})


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
bronze_schema = StructType([
    StructField("temperature", IntegerType()),
    StructField("humidity", IntegerType()),
    StructField("city", StringType()),
    StructField("timestamp", TimestampType())
])

In [0]:
df = (spark.readStream
      .format("json")
      .schema(bronze_schema)
      .load("abfss://bronze@streamingsa.dfs.core.windows.net/EventHub")
      .withColumn("timestamp", to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss"))
      .select("temperature", "humidity", "city", "timestamp")
)

(df.writeStream
 .option("checkpointLocation", "abfss://checkpoint@streamingsa.dfs.core.windows.net/checkpoint")
 .outputMode("append")
 .format("parquet")
 .option("path", "abfss://silver@streamingsa.dfs.core.windows.net/EventHub/weather")
 .start()
 )